In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha6"

In [ ]:
open Deedle
open Plotly.NET
open System
open System.Net.Http
open System.Net.Http.Json
open System.Text.Json
open System.Text.Json.Serialization

let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

In [ ]:
let baseUrl = "http://localhost:5000"

let client = new HttpClient()
client.BaseAddress <- Uri(baseUrl)

client.GetAsync("/")
|> Async.AwaitTask
|> Async.RunSynchronously

In [ ]:
type AlternativeLoadForecast =
  { Forecast: float32 array
    LowerBound: float32 array
    UpperBound: float32 array }

let from = DateTime.Parse("2019-03-10T00:00:00")
let horizon = 24 * 10


let response =
    client.PostAsJsonAsync("/ssa-forecast", {| From = from; Horizon = horizon |})
    |> Async.AwaitTask
    |> Async.RunSynchronously

let forecastedValues, lower, upper =
    response.Content.ReadFromJsonAsync<AlternativeLoadForecast>()
    |> Async.AwaitTask
    |> Async.RunSynchronously
    |> fun forecast -> forecast.Forecast, forecast.LowerBound, forecast.UpperBound

forecastedValues
|> Array.take 10

In [ ]:
let keys, actualValues =
    data?Value
    |> Series.observations
    |> Seq.skipWhile (fun (k, _) -> k <> from)
    |> Seq.take horizon
    |> Seq.unzip

let predChart =
    Seq.zip keys forecastedValues
    |> fun xy -> Chart.Range(xy,
                             lower,
                             upper,
                             mode = StyleParam.Mode.Lines,
                             Color = Colors.toWebColor Colors.Table.Office.blue,
                             RangeColor = Colors.toWebColor Colors.Table.Office.lightBlue)
    |> Chart.withTraceName "Forecast_CI"

let actualChart =
    Seq.zip keys actualValues
    |> fun xy -> Chart.Line(xy,
                            Color = Colors.toWebColor Colors.Table.Office.orange,
                            UseWebGL = true,
                            Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine